0. This notebook is the continuation to the preprocessing.ipynb notebook.

1. Import all neccessary modules 

In [1]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

2. Import X and y

In [2]:
import torchvision
from torchvision import transforms
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image

Code that loads the preprocessed npy arrays from the kosti dataset, extract frames and mscoco rows. Save for processing! This array contains all 3 variables at once.

In [36]:
train_arr = np.load('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/kosti/npyfiles/emotic_pre/train_cont_arr.npy')

In [62]:
train = pd.read_csv('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/kosti/mat2csv/emotic_pre/train.csv')

In [64]:
idx = train.index[train['Folder']=='mscoco/images']

In [67]:
np_idx = np.flatnonzero(train['Folder']=='mscoco/images')

In [73]:
frames_idx = np.flatnonzero(train['Folder']=='framesdb/images')

In [69]:
np_idx.shape

(14276,)

In [70]:
t_np = train_arr[np_idx]

In [74]:
t_frames = train_arr[frames_idx]

In [79]:
train_all = np.concatenate((t_np,t_frames))

In [82]:
np.save('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/train_con', train_all)

In [83]:
test = np.load('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/train_con.npy')

In [66]:
type(idx)

pandas.core.indexes.numeric.Int64Index

In [60]:
train_arr[0] == y_arr[0]

array([False, False, False])

In [61]:
y_arr[0]

array(['[5, 3, 9]'], dtype=object)

In [34]:
arousal = pd.read_csv('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/All_ratings_train.csv')

UnicodeDecodeError: 'utf-8' codec can't decode byte 0x93 in position 0: invalid start byte

In [4]:
y = y.iloc[:,[1]]

In [5]:
y

,0
0,"[5, 3, 9]"
1,"[6, 4, 7]"
2,"[7, 8, 8]"
3,"[8, 9, 8]"
4,"[7, 9, 10]"
...,...
21619,"[6, 6, 6]"
21620,"[5, 7, 4]"
21621,"[5, 4, 6]"
21622,"[6, 7, 6]"


In [6]:
y_8000_rows = y[0:100]

In [8]:
y_arr = y_8000_rows.to_numpy()

In [40]:
y_arr.shape

(100, 1)

In [9]:
os.chdir('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis')

NEW APPROACH:

Fit CNN on X

1. Read `n-rows`from X.

In [10]:
chunk = 10
i=0
X=[]
import pandas as pd
for df in pd.read_csv('X_matrix.csv',chunksize = chunk):
    X.append(df)
    i=i+1
    print('Processed {i}/10 chunks'.format(i=i))
    if i == 10:
        break



Processed 1/10 chunks
Processed 2/10 chunks
Processed 3/10 chunks
Processed 4/10 chunks
Processed 5/10 chunks
Processed 6/10 chunks
Processed 7/10 chunks
Processed 8/10 chunks
Processed 9/10 chunks
Processed 10/10 chunks


In [11]:
X = pd.concat(X)

In [12]:
del X [X.columns[0]]

In [13]:
X_arr = X.to_numpy()

In [14]:
X_arr = np.reshape(X_arr,(100,224,224))

In [15]:
X_arr.shape

(100, 224, 224)

Adding new dimension for CNN

In [16]:
X_ = X_arr[:,:,np.newaxis]

In [17]:
X_ = np.swapaxes(X_,2,1)

In [18]:
X_.shape

(100, 1, 224, 224)

In [19]:
X_train,X_test,y_train,y_test = train_test_split(X_,y_arr,test_size=0.2,random_state=42)

In [33]:
y_train=np.vstack(y_train).astype(np.float)
y_test=np.vstack(y_test).astype(np.float)

/tmp/ipykernel_100/3428171516.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_train=np.vstack(y_train).astype(np.float)


ValueError: could not convert string to float: '[10, 6, 8]'

Converting X to appropriate Datatype...

In [27]:
X_tensor_train = torch.from_numpy(X_train)
X_tensor_test = torch.from_numpy(X_test)
y_tensor_train = torch.from_numpy(y_train)
y_tensor_test = torch.from_numpy(y_test)

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.

In [20]:
X_tensor_train = torch.tensor(X_tensor_train,dtype = torch.float32)
X_tensor_test = torch.tensor(X_tensor_test,dtype = torch.float32)
y_tensor_train = y_tensor_train.type(torch.LongTensor)
y_tensor_test = y_tensor_test.type(torch.LongTensor)

/tmp/ipykernel_142/3712994864.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor_train = torch.tensor(X_tensor_train,dtype = torch.float32)
/tmp/ipykernel_142/3712994864.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_tensor_test = torch.tensor(X_tensor_test,dtype = torch.float32)


Pytorch needs the true labels to be in the range N-1. Thus, substract 1 one from each label.

In [21]:
y_tensor_train = y_tensor_train-1
y_tensor_test = y_tensor_test-1

In [22]:
print('training shape X is {}, test shape X is{}'.format(X_tensor_train.shape,X_tensor_test.shape))

training shape X is torch.Size([80, 1, 224, 224]), test shape X istorch.Size([20, 1, 224, 224])


Setup custom dataset class

class CustomDataset(Dataset):
    def __init__(self,X_path, y_path):#, y_path):
        df_x = pd.read_csv(X_path)
        df_y = pd.read_csv(y_path)#'/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/Arousal.csv')
        del X [X.columns[0]]
        X = X.to_numpy()
        X = np.reshape(X,(100,224,224))
        X = X[:,:,np.newaxis]
        X = np.swapaxes(X,2,1)
        y = df_y.iloc[:,[1]]
        
        self.X_train = X
        self.y_train = y.to_numpy()
        
        
    def __len__(self):
        return len(self.y_train)#X)
    
    def __getitem__(self, idx):
        X_train = self.X[idx]
        y_train= self.y_train[idx]
        return X_train, y_train


In [23]:
class SimpleDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
        
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        X = self.X[idx]
        y = self.y[idx]
        return X, y

In [24]:
import torch.nn as nn

In [25]:
dataset_train = SimpleDataset(X_tensor_train,y_tensor_train)
dataloader_train = DataLoader(dataset_train, batch_size=32, shuffle=True)

dataset_test = SimpleDataset(X_tensor_test,y_tensor_test)
dataloader_test = DataLoader(dataset_test, batch_size = 32,shuffle=True)
criterion = nn.CrossEntropyLoss()

dataset = CustomDataset('/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/X_matrix.csv','/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/Arousal.csv')

dataset = CustomDataset(X_path ='/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/X_matrix.csv', y_path='/mnt/c/Users/janos/OneDrive/Desktop/Master_thesis/Arousal.csv')
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

Setting up the CNN.

In [26]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.fc1 = nn.Linear(774400, 128)
        self.fc2 = nn.Linear(128, 10)
        self.pool = nn.MaxPool2d(2)
        
    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        return x


In [27]:
SalienceNet = CNN()

In [37]:
def analysis(net, loss_fn, dataloader_train,dataloader_test,
          epoch=100, learning_rate=0.01):
    optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)#,weight_decay=1e-4)
    train_loss = []
    acc=[]
    for i in range(epoch):   
        net.train()
        correct, total = 0, 0
        loss_count = 0
        for data, label in dataloader_train:
            data, label = data.to('cpu'), label.to('cpu')
        #make prediction
            output = net(data)

            # Zero out the gradients
            optimizer.zero_grad()

            # Calculate loss.
            loss = criterion(output,torch.max(label, 1)[0])

            # Backpropagation and gradient update.
            loss.backward()# Calculate gradients. #retrain_graph=True

            optimizer.step() # Apply gradient udpate.

            loss_count += loss.item()
            _, predicted = torch.max(output, 1)
            print(predicted, torch.max(label,1)[0])
            total += label.size(0)
            correct += (predicted == torch.max(label,1)[0]).sum().item()
          #  if (i + 1) % (epoch // 5) == 0:
           #     print(f'iteration {i + 1}/{epoch} | loss: {loss.item():.3f}')
    acc.append(correct/total)
    train_loss.append(loss_count/len(dataloader_train))
    
    net.eval()
    test_loss = []
    test_acc = []
    correct_test, total_test = 0,0
    for data, labels in dataloader_test:
        data, labels = data.to('cpu'), labels.to('cpu')
        
        outputs = net(data)
        _, pred_test = torch.max(outputs,1)
        total_test += labels.size(0)
        print(total_test)
        correct_test += (pred_test == torch.max(labels,1)[0]).sum().item()
        print(correct_test)
        test_l = criterion(outputs,torch.max(labels, 1)[0])
        test_loss.append(test_l.item())
    test_acc.append(correct_test/total_test)
    print(acc,test_acc)
    return acc,total,correct,train_loss,test_acc


In [29]:
import matplotlib.pyplot as plt

In [38]:
np.random.seed(1)
torch.manual_seed(1)

acc,total,correct,train_loss,test_acc=analysis(net=SalienceNet,loss_fn=criterion,dataloader_train=dataloader_train,
                  dataloader_test=dataloader_test,epoch=2)



20
3
[0.175] [0.15]


In [39]:
train_loss

[2.505991776784261]

In [41]:
i=[]

i.append(6/3)
i

[2.0]

In [81]:
total,correct = 0,0
i=0
loss = []
while i < 5:
    for x,y in dataloader_train:
        output = SalienceNet(x)
        _, predicted = torch.max(output, 1)
        #predicted = torch.tensor()
        total += y.size(0)
        correct += (predicted == torch.max(y,1)[0]).sum().item()
        print(predicted,torch.transpose(y,1,0))
        y_I = torch.max(y,1)[0]
        i=i+1
loss.append(correct/total)
print(loss)

tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[5, 6, 3, 5, 6, 5, 5, 5, 5, 5, 5, 1, 4, 6, 6, 4, 5, 4, 4, 3, 6, 4, 6, 6,
         5, 5, 4, 3, 4, 5, 4, 4]])
tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[6, 5, 4, 3, 6, 7, 4, 4, 4, 5, 5, 9, 6, 7, 5, 4, 6, 6, 6, 6, 6, 7, 2, 5,
         6, 6, 7, 5, 4, 7, 5, 6]])
tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]) tensor([[4, 1, 5, 6, 3, 5, 6, 7, 5, 6, 5, 2, 4, 5, 5, 5]])
tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[6, 5, 5, 5, 5, 3, 7, 3, 5, 4, 6, 5, 3, 5, 5, 7, 4, 9, 6, 7, 3, 4, 6, 5,
         1, 7, 5, 6, 5, 4, 4, 5]])
tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4,
        4, 4, 4, 4, 4, 4, 4, 4]) tensor([[6, 6, 5, 6, 6, 4, 5, 7, 7, 5, 6, 5, 4, 4, 6, 1, 4, 5, 6, 5, 6, 6, 2, 5,
   

In [77]:
_, predicted

(tensor([0.1476, 0.1476, 0.1476, 0.1476, 0.1476, 0.1476, 0.1476, 0.1476, 0.1476,
         0.1476, 0.1476, 0.1476, 0.1476, 0.1476, 0.1476, 0.1476],
        grad_fn=<MaxBackward0>),
 tensor([4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4]))

In [83]:
y_I

tensor([4, 4, 6, 4, 5, 6, 6, 5, 5, 4, 5, 5, 4, 4, 6, 2])

In [88]:
total

160

In [51]:
output

tensor([[-65.9044,  44.7948, -50.0664,  22.9917,  63.5114, -65.3450,  30.7097,
         130.8091, -44.3176,   9.8188],
        [-66.7599,  45.2335, -50.6327,  23.2442,  64.3278, -66.1742,  31.0289,
         132.4245, -45.0380,   9.9740],
        [-67.2260,  45.4627, -50.9415,  23.4104,  64.8238, -66.7101,  31.0802,
         133.4065, -45.5668,  10.1352],
        [-66.6873,  45.1616, -50.5517,  23.2404,  64.2822, -66.0941,  30.9580,
         132.2808, -45.0118,  10.0283],
        [-66.2129,  44.9116, -50.2500,  23.0817,  63.8136, -65.5273,  30.9220,
         131.2758, -44.4338,   9.8651],
        [-67.0469,  45.4192, -50.8229,  23.3484,  64.6289, -66.5578,  31.0135,
         133.0656, -45.3902,  10.1040],
        [-66.3755,  45.0062, -50.3480,  23.1280,  63.9697, -65.7896,  30.8553,
         131.6962, -44.7400,   9.9400],
        [-67.0683,  45.3656, -50.8393,  23.3909,  64.6840, -66.6029,  30.9773,
         133.1539, -45.5212,  10.1378],
        [-66.3048,  44.9367, -50.3148,  23.1251,

In [ ]:
correct

In [ ]:
total

In [ ]:
5/40

In [ ]:
for idx, (x,y) in enumerate(dataloader_train):
    print(y)
    

In [ ]:
acc,total,correct

In [ ]:
plt.plot(train_loss)
plt.xlabel('Iterations of gradient descent')
plt.ylabel('Cross entropy')
plt.show()